<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=4fc46589d98e96b060a9626305fd32ba481214e8dac70c8aa51cd2d845d22361
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-23 11:12:26
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.42 C
-------------------
Today PnL: 13.18 K (0.09%)
Current PnL: -25.53 L (-16.49%)
CY Booked + Current PnL: -11.24 L (-7.26%)
-------------------
Total profit:  1.65 L
Total loss:  -27.18 L
-------------------
Total Booked + Current PnL: 15.74 L (12.32%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.1%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 90.16 L (63.71%)
Deployed:  1.28 C
Current:  1.42 C
CAGR/XIRR %: 5.62%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,-1.19,-18.99,23.56,0.09,19234.0,-19138.0,81639.0,89.54,39.0,M-SC,2.39,253.0,-1.00,0.58,7.26,OX40N,NTT,DURABLES
82,VOLTAS,1530.00,-0.48,8.07,10.76,19.69,22296.0,15468.0,207210.0,-2.46,53.0,X-MC,2.47,78.0,0.69,1.47,15.41,XY25,NTT,AC
50,MASFIN,398.61,1.09,-2.20,24.80,22.05,23763.0,-2160.0,95820.0,-15.54,54.0,H-SC,6.33,164.0,-0.09,0.68,39.28,XR,ATH,FINANCE
43,ITC,452.00,0.57,-0.74,11.60,10.78,27671.0,-1773.0,238545.0,-40.74,54.0,X-LC,2.10,5.0,-0.06,1.69,4.46,X40,NTT,FMCG
52,NESTLEIND,1377.00,0.00,10.83,9.53,21.40,28036.0,28759.0,294185.0,2.90,56.0,X-LC,5.48,12.0,1.03,2.09,18.20,XY25,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COFFEEDAY,80.00,3.37,-39.40,123.21,35.27,84784.0,-44736.0,68813.0,-53.72,51.0,L-SC,9.72,270.0,-0.53,0.49,68.42,XR,NTT,HOTELS
17,COALINDIA,484.83,3.25,-1.93,21.57,19.23,31828.0,-2904.0,147556.0,15.50,72.0,L-LC,2.10,179.0,-0.09,1.05,18.93,XY25,ATH,MINING
79,VAIBHAVGBL,521.00,2.84,-25.46,115.69,60.77,157609.0,-46541.0,136234.0,57.02,52.0,H-SC,4.70,159.0,-0.30,0.97,22.69,XR,NTT,JEWELLERY
58,RECLTD,446.00,2.74,-4.24,26.08,20.74,50742.0,-8608.0,194562.0,26.46,55.0,M-MC,8.50,189.0,-0.17,1.38,5.93,XY25,NTT,FINANCE
74,TITAGARH,1548.00,2.41,-21.28,85.13,45.74,162294.0,-51535.0,190642.0,4.95,58.0,H-SC,6.35,174.0,-0.32,1.35,23.99,XY24,NTT,ENGINEERING


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RAJESHEXPO,518.00,-3.20,-56.86,131.97,0.07,78092.0,-78003.0,59174.0,2069.19,71.0,L-SC,3.63,268.0,-1.00,0.42,47.17,OX40N,NTT,JEWELLERY
22,DIXON,18931.72,-3.19,-14.46,47.39,26.07,79134.0,-28236.0,166985.0,-58.39,33.0,X-MC,11.39,56.0,-0.36,1.19,4.00,X40N,ATH,IT
6,ASIANTILES,137.00,-3.05,-5.17,91.58,81.67,81468.0,-4852.0,88958.0,8005.56,65.0,L-SC,12.09,271.0,-0.06,0.63,72.27,XR,NTT,CERAMICS
67,SONACOMS,806.63,-1.53,-15.94,66.01,39.54,56130.0,-16127.0,85032.0,-34.23,48.0,M-SC,3.84,220.0,-0.29,0.60,19.90,AR,ATH,AUTO
57,RAJOOENG,143.10,-1.48,-38.10,125.53,39.61,79649.0,-39050.0,63450.0,-58.14,32.0,H-SC,30.51,135.0,-0.49,0.45,0.00,AR,ATH,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.80,-1.42,110.90,107.91,161416.0,-2093.0,145551.0,-19.74,56.0,M-SC,11.90,216.0,-0.01,1.03,8.53,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-0.37,-16.48,19.75,0.02,28795.0,-28760.0,145799.0,122.10,41.0,M-SC,12.04,234.0,-1.00,1.04,26.82,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-1.19,-18.99,23.56,0.09,19234.0,-19138.0,81639.0,89.54,39.0,M-SC,2.39,253.0,-1.00,0.58,7.26,OX40N,NTT,DURABLES
72,TATAELXSI,9161.00,-0.15,-21.45,68.08,32.03,70504.0,-28275.0,103560.0,-15.03,63.0,H-SC,9.08,148.0,-0.40,0.74,16.00,OX40N,NTT,IT
66,SIS,528.00,-0.72,-24.05,59.66,21.26,50310.0,-26704.0,84328.0,1989.64,50.0,H-SC,4.73,166.0,-0.53,0.60,13.94,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,-0.37,11.42,54.61,72.27,103102.0,19349.0,188797.0,-4.26,75.0,M-LC,3.68,99.0,0.19,1.34,18.3,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-0.37,11.42,54.61,72.27,103102.0,19349.0,188797.0,-4.26,75.0,M-LC,3.68,99.0,0.19,1.34,18.30,XR,NTT,IT
25,FINCABLES,1641.55,-0.80,-1.42,110.90,107.91,161416.0,-2093.0,145551.0,-19.74,56.0,M-SC,11.90,216.0,-0.01,1.03,8.53,OX40N,ATH,CABLES
38,INDIAMART,4810.62,0.64,-4.08,115.51,106.72,136656.0,-5029.0,118307.0,-52.54,39.0,H-SC,4.26,139.0,-0.04,0.84,18.35,AR,ATH,MISC
6,ASIANTILES,137.00,-3.05,-5.17,91.58,81.67,81468.0,-4852.0,88958.0,8005.56,65.0,L-SC,12.09,271.0,-0.06,0.63,72.27,XR,NTT,CERAMICS
85,ZYDUSLIFE,1286.17,0.46,-2.73,39.44,35.63,80398.0,-5724.0,203850.0,-16.30,43.0,H-MC,2.40,119.0,-0.07,1.45,13.49,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,AWL,485.00,-0.70,-24.04,102.08,53.50,233967.0,-72551.0,229200.0,-63.25,33.0,X-MC,5.40,58.0,-0.31,1.63,1.80,XY24,NTT,FMCG
9,BAJAJHFL,181.50,0.16,-17.72,88.96,55.47,178069.0,-43118.0,200168.0,-27.18,33.0,X-MC,15.19,64.0,-0.24,1.42,2.56,X40N,ATH,FINANCE
22,DIXON,18931.72,-3.19,-14.46,47.39,26.07,79134.0,-28236.0,166985.0,-58.39,33.0,X-MC,11.39,56.0,-0.36,1.19,4.00,X40N,ATH,IT
34,HONAUT,58357.33,-0.34,-17.76,74.28,43.32,99491.0,-28932.0,133940.0,-29.29,33.0,X-SC,5.64,90.0,-0.29,0.95,2.84,X40N,ATH,ELECTRICAL
11,BATAINDIA,2096.00,-0.10,-38.90,121.30,35.20,95358.0,-50057.0,78613.0,3.10,34.0,X-SC,16.87,92.0,-0.52,0.56,0.84,X40,NTT,FOOTWEAR


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,950.0,-0.05,-0.18,24.70,24.48,62094.0,-462.0,251394.0,-80.87,49.0,X-SC,1.17,86.0,-0.01,1.78,22.91,X40N,NTT,MISC
12,BERGEPAINT,680.0,0.51,-2.93,24.79,21.14,54708.0,-6655.0,220684.0,-9.80,46.0,X-MC,1.18,74.0,-0.12,1.57,24.39,XY24,NTT,PAINTS
20,DABUR,735.0,-0.12,-3.59,49.06,43.70,117329.0,-8914.0,239154.0,-11.62,35.0,X-MC,2.08,72.0,-0.08,1.70,10.92,XY24,BTT,FMCG
43,ITC,452.0,0.57,-0.74,11.60,10.78,27671.0,-1773.0,238545.0,-40.74,54.0,X-LC,2.10,5.0,-0.06,1.69,4.46,X40,NTT,FMCG
78,UNITDSPR,1644.0,1.19,6.05,13.87,20.77,34441.0,14172.0,248316.0,-3.23,55.0,X-MC,2.16,62.0,0.41,1.76,13.39,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-1.03,-25.86,121.42,64.15,214197.0,-61541.0,176410.0,-56.75,38.0,X-SC,3.80,82.0,-0.29,1.25,0.72,XY24,BTT,CEMENT
11,BATAINDIA,2096.00,-0.10,-38.90,121.30,35.20,95358.0,-50057.0,78613.0,3.10,34.0,X-SC,16.87,92.0,-0.52,0.56,0.84,X40,NTT,FOOTWEAR
19,COLPAL,3726.84,0.09,-19.88,76.61,41.51,161662.0,-52345.0,211020.0,-4.54,41.0,X-MC,7.72,61.0,-0.32,1.50,1.09,XY25,ATH,FMCG
59,RELAXO,1176.00,0.08,-47.85,192.03,52.31,145382.0,-69452.0,75708.0,-44.44,42.0,X-SC,7.27,91.0,-0.48,0.54,1.34,X40N,NTT,FOOTWEAR
8,AWL,485.00,-0.70,-24.04,102.08,53.50,233967.0,-72551.0,229200.0,-63.25,33.0,X-MC,5.40,58.0,-0.31,1.63,1.80,XY24,NTT,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-1.15,-29.32,102.10,42.84,46911.0,-19060.0,45946.0,-53.72,46.0,X-SC,34.26,83.0,-0.41,0.33,3.95,XY24,NTT,MISC
59,RELAXO,1176.00,0.08,-47.85,192.03,52.31,145382.0,-69452.0,75708.0,-44.44,42.0,X-SC,7.27,91.0,-0.48,0.54,1.34,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.10,-38.90,121.30,35.20,95358.0,-50057.0,78613.0,3.10,34.0,X-SC,16.87,92.0,-0.52,0.56,0.84,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,1.01,-1.47,25.93,24.08,32433.0,-1870.0,125080.0,-9.50,46.0,X-SC,6.14,89.0,-0.06,0.89,17.85,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.34,-17.76,74.28,43.32,99491.0,-28932.0,133940.0,-29.29,33.0,X-SC,5.64,90.0,-0.29,0.95,2.84,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-0.67,-9.19,32.92,20.71,113074.0,-34756.0,343481.0,-21.68,68.0,X-LC,3.46,1.0,-0.31,2.44,14.77,X40,ATH,IT
41,INFY,1972.00,-1.11,13.75,18.02,34.24,64134.0,43014.0,355902.0,-11.34,69.0,X-LC,5.82,2.0,0.67,2.53,23.17,X40,NTT,IT
75,TMPV,600.00,1.24,-25.48,65.02,22.98,109459.0,-57551.0,168347.0,-23.08,54.0,X-LC,6.41,3.0,-0.53,1.20,5.88,XY24,NTT,AUTO
81,VBL,671.64,-1.04,-2.92,39.75,35.67,121883.0,-9219.0,306623.0,-14.52,56.0,X-LC,3.61,4.0,-0.08,2.18,10.53,X40N,ATH,FMCG
43,ITC,452.00,0.57,-0.74,11.60,10.78,27671.0,-1773.0,238545.0,-40.74,54.0,X-LC,2.10,5.0,-0.06,1.69,4.46,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-0.19,-36.90,110.59,32.89,53881.0,-28486.0,48721.0,-701.92,57.0,L-MC,6.84,259.0,-0.53,0.35,34.17,XR,NTT,BANKS
56,RAJESHEXPO,518.00,-3.20,-56.86,131.97,0.07,78092.0,-78003.0,59174.0,2069.19,71.0,L-SC,3.63,268.0,-1.00,0.42,47.17,OX40N,NTT,JEWELLERY
6,ASIANTILES,137.00,-3.05,-5.17,91.58,81.67,81468.0,-4852.0,88958.0,8005.56,65.0,L-SC,12.09,271.0,-0.06,0.63,72.27,XR,NTT,CERAMICS
50,MASFIN,398.61,1.09,-2.20,24.80,22.05,23763.0,-2160.0,95820.0,-15.54,54.0,H-SC,6.33,164.0,-0.09,0.68,39.28,XR,ATH,FINANCE
13,BSOFT,831.70,1.53,-17.14,81.97,50.79,95534.0,-24105.0,116548.0,6.30,74.0,H-SC,7.65,151.0,-0.25,0.83,35.70,XR,ATH,IT


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,1.53,-17.14,81.97,50.79,95534.0,-24105.0,116548.0,6.30,74.0,H-SC,7.65,151.0,-0.25,0.83,35.70,XR,ATH,IT
56,RAJESHEXPO,518.00,-3.20,-56.86,131.97,0.07,78092.0,-78003.0,59174.0,2069.19,71.0,L-SC,3.63,268.0,-1.00,0.42,47.17,OX40N,NTT,JEWELLERY
61,ROUTE,2227.21,0.31,-45.46,209.98,69.05,152378.0,-60498.0,72568.0,-55.87,56.0,H-SC,20.97,140.0,-0.40,0.52,11.92,SR,ATH,IT
84,WIPRO,420.00,-0.37,11.42,54.61,72.27,103102.0,19349.0,188797.0,-4.26,75.0,M-LC,3.68,99.0,0.19,1.34,18.30,XR,NTT,IT
6,ASIANTILES,137.00,-3.05,-5.17,91.58,81.67,81468.0,-4852.0,88958.0,8005.56,65.0,L-SC,12.09,271.0,-0.06,0.63,72.27,XR,NTT,CERAMICS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.38
1,25,44.84
2,50,76.58


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.94
MC    30.76
LC    24.34
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.79
X40      23.01
X40N     12.98
XR        9.93
XY25      9.12
AR        8.89
OX40N     7.56
SR        1.01
MH        0.75
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.24
H-SC    23.80
X-LC    20.84
M-SC    11.60
X-SC     8.00
H-MC     4.79
L-SC     1.54
M-MC     1.38
M-LC     1.34
H-LC     1.11
L-LC     1.05
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.23,-7.57,42.16
IT,13.21,-13.21,70.13
FINANCE,9.54,-18.52,70.07
MISC,7.26,-26.98,78.92
ELECTRICAL,5.95,-11.95,52.96
PAINTS,5.78,-13.44,30.29
INSURANCE,4.75,-1.35,36.04
PHARMA,4.06,-2.76,35.11
AUTO,2.86,-30.01,75.08


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3149677.0,21
AR,1318962.0,10
XR,1265859.0,13
X40,982501.0,14
X40N,857768.0,9
OX40N,727067.0,10
XY25,356686.0,6
SR,278600.0,2
MH,79044.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3626735.0,25
M-SC,1424837.0,15
X-MC,1413512.0,16
X-LC,792945.0,11
X-SC,792608.0,8
H-MC,404054.0,3
L-SC,244344.0,3
M-LC,103102.0,1
H-LC,77576.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1265349.0      6
           AR         903387.0      5
M-SC       XY24       808646.0      6
H-SC       XR         769311.0      7
X-MC       X40        487553.0      7
           XY24       406004.0      3
           X40N       335997.0      4
H-SC       OX40N      331044.0      4
M-SC       OX40N      317931.0      5
X-SC       X40N       306967.0      3
X-LC       X40        302848.0      5
X-SC       XY24       293541.0      3
H-SC       SR         278600.0      2
X-LC       X40N       214804.0      2
H-MC       AR         213052.0      2
X-SC       X40        192100.0      2
H-MC       XY24       191002.0      1
X-LC       XY24       185135.0      2
X-MC       XY25       183958.0      2
M-SC       XR         173313.0      2
L-SC       XR         166252.0      2
M-SC       AR         124947.0      2
M-LC       XR         103102.0      1
X-LC       XY25        90158.0      2
H-SC       MH          79044.0      1
L-SC       OX40N       78092.0      1
H-LC       AR          77576.0      1
L-MC       XR          53881.0      1
M-MC       XY25        50742.0      1
L-LC       XY25        31828.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 56.0 seconds
